## Astr-119-Final-Project

Importing numpy and sep.

In [ ]:
import numpy as np
import sep

Importing matplotlib, rcParams, %matplotlib inline, and astropy.io.fits. Then using rcParams, from matplotlib, to increase the size of all the figures that will follow along in the code.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

from astropy.io import fits
rcParams['figure.figsize'] = [10., 8.]

Using astropy to directly download the fits image from the online source.

In [ ]:
from astropy.utils.data import download_file
image_file = download_file('https://raw.githubusercontent.com/kbarbary/sep/master/data/image.fits', cache=True)

Reading and opening the contents of the fits image.

In [ ]:
hdu_list = fits.open(image_file)
hdu_list.info()

Storing the data of the image in a 2-D numpy array.

In [ ]:
image_data = hdu_list[0].data

Showing the dimensions of the numpy array onto the screen.

In [ ]:
print(type(image_data))
print(image_data.shape)

Now that the data is stored into an array, closing the fits image.

In [ ]:
hdu_list.close

Defining and storing the variables for the mean and standrard deviations of the image,
displaying the image to screen using matplotlib in terms of the mean and standard deviation, 
and saving the figure as a png file to the desktop.

In [ ]:
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig('AstroSourceImage.png',bbox_inches="tight",dpi=300)

Displaying some of the basic statistics of the image to screen.

In [ ]:
print('Minimum:', np.min(image_data))
print('Maximum:', np.max(image_data))
print('Mean:', np.mean(image_data))
print('StandardDeviation:', np.std(image_data))

Measuring the varying background in space in the fits image.

In [ ]:
bkgrd = sep.Background(image_data)

Getting a entire mean and noise value of the image to screen.

In [ ]:
print('GlobalMean', bkgrd.globalback)
print('GlobalNoise', bkgrd.globalrms)

Analyzing the background as a 2-D numpy array.

In [ ]:
bkgrd_image = bkgrd.back()

Displaying the background image to screen via matplotlib, and saving it to the deskstop as a png file.

In [ ]:
plt.imshow(bkgrd_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('AstroSourceBkgrdImage.png', bbox_inches='tight',dpi=300)

Analyzing the background noise as 2-D numpy array.

In [ ]:
bkgrd_rms = bkgrd.rms()

Displaying the noise image to the screen via matplotlib, and saving it to the desktop as a png file.

In [ ]:
plt.imshow(bkgrd_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('AstroSourceBkgrdRms.png', bbox_inches='tight',dpi=300)

Subtracting the background so objects can be detected clearly.

In [ ]:
data_sub = image_data - bkgrd

Showing the subtraction data to the screen.

In [ ]:
print(data_sub)

Using numpy and sep to detect the number of objects, setting the detection threshold to 1.5., and defining the error.

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkgrd.globalrms)

Showing how many objects have been detected.

In [ ]:
len(objects)

Importing Ellipse from matplotlib.

In [ ]:
from matplotlib.patches import Ellipse

Poltting and showing the background-subtracted image, setting the center and color for each ellipse around the objects, and saving the image results as a png file.

In [ ]:
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
               width=6*objects['a'][i],
               height=6*objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('AstroSourceObjects.png', bbox_inches='tight',dpi=300)

Displaying the fields for the 'objects' command.

In [ ]:
objects.dtype.names

Permforing circular aperture photometry while using 3 pixel radius at object locations, and storing them in 1-D numpy arrays per object.

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                    3.0, err=bkgrd.globalrms, gain=1.0)

Showing the results for the first ten objects detected.

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))